<a href="https://colab.research.google.com/github/b-schoen/gpt_from_scratch/blob/main/colab/arena_4_0_prereq_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [0.0] - Prerequisites

**This is my own copy of the ARENA prereq exercises from https://arena3-chapter0-fundamentals.streamlit.app/%5B0.0%5D_Prerequisites**

Colab: [exercises](https://colab.research.google.com/drive/1vuQOB2Gd7OcfzH2y9djXm9OdZA_DcxYz) | [solutions](https://colab.research.google.com/drive/1sUqHDcZUJl67hVEWWXVZoB0av7qwOvMw)

ARENA 3.0 [Streamlit page](https://arena3-chapter0-fundamentals.streamlit.app/[0.0]_Prerequisites)

Please send any problems / bugs on the `#errata` channel in the [Slack group](https://join.slack.com/t/arena-la82367/shared_invite/zt-1uvoagohe-JUv9xB7Vr143pdx1UBPrzQ), and ask any questions on the dedicated channels for this chapter of material.



<img src="https://raw.githubusercontent.com/callummcdougall/computational-thread-art/master/example_images/misc/pres2.png" width="350">

This notebook contains important prerequisites for the rest of the material in this chapter.

The first section **1️⃣ Core Concepts / Knowledge** is a long list of important concepts and libraries that you should be familiar with before starting the rest of the material.

The second section **2️⃣ Einops, Einsum & Tensor Manipulation** provides some coding exercises to help you get familiar with the `einops` and `einsum` libraries, as well as some exercises on indexing and other aspects of tensor manipulation. This section contains a large number of exercises. You should generally feel free to skim through them (skipping some or most) if you feel comfortable with the basic ideas.

# Setup (don't read, just run!)

In [1]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    # Install packages
    %pip install einops
    %pip install jaxtyping

    # Code to make sure output widgets display
    from google.colab import output
    output.enable_custom_widget_manager()

    # Code to download the necessary files (e.g. solutions, test funcs)
    import os, sys
    if not os.path.exists("chapter0_fundamentals"):
        !wget https://github.com/callummcdougall/ARENA_3.0/archive/refs/heads/main.zip
        !unzip /content/main.zip 'ARENA_3.0-main/chapter0_fundamentals/exercises/*'
        os.remove("/content/main.zip")
        os.rename("ARENA_3.0-main/chapter0_fundamentals", "chapter0_fundamentals")
        os.rmdir("ARENA_3.0-main")
        sys.path.insert(0, "chapter0_fundamentals/exercises")

    # Clear output
    from IPython.display import clear_output
    clear_output()
    print("Imports & installations complete!")

else:
    from IPython import get_ipython
    ipython = get_ipython()
    ipython.run_line_magic("load_ext", "autoreload")
    ipython.run_line_magic("autoreload", "2")

Imports & installations complete!


In [2]:
import sys
import math
import numpy as np
import einops
import torch as t
from pathlib import Path

# Get file paths to this set of exercises
exercises_dir = Path("chapter0_fundamentals/exercises")
section_dir = exercises_dir / "part0_prereqs"

from plotly_utils import imshow, line, bar
from part0_prereqs.utils import display_array_as_img, display_soln_array_as_img
import part0_prereqs.tests as tests

MAIN = __name__ == "__main__"

# 1️⃣ Core Concepts / Knowledge

## Linear Algebra Intuitions

[Core Concepts / Knowledge](https://arena3-chapter0-fundamentals.streamlit.app/%5B0.0%5D_Prerequisites)

* SVD
 * [Six (and a half) intuitions for SVD](https://www.lesswrong.com/posts/iupCxk3ddiJBAJkts/six-and-a-half-intuitions-for-svd)
 * [The Singular Value Decompositions of Transformer Weight Matrices are Highly Interpretable](https://www.lesswrong.com/posts/mkbGjzxD8d8XqKHzA/the-singular-value-decompositions-of-transformer-weight)
* KL Divergence
 * [Six (and a half) intuitions for KL divergence](https://www.lesswrong.com/posts/no5jDTut5Byjqb4j5/six-and-a-half-intuitions-for-kl-divergence)

# 2️⃣ Einops, Einsum & Tensor Manipulation

> ### Learning objectives
>
> - Understand the basics of Einstein summation convention
> - Learn how to use `einops` to perform basic tensor rearrangement, and `einsum` to to perform standard linear algebra operations on tensors

Note - this section contains a large number of exercises. You should generally feel free to skim through them (skipping some or most) if you feel comfortable with the basic ideas.

## Reading

* Read about the benefits of the `einops` library [here](https://www.blopig.com/blog/2022/05/einops-powerful-library-for-tensor-operations-in-deep-learning/).
* If you haven't already, then review the [Einops basics tutorial](https://einops.rocks/1-einops-basics/) (up to the "fancy examples" section).
* Read [einsum is all you need](https://rockt.github.io/2018/04/30/einsum) (or [watch it](https://www.youtube.com/watch?v=pkVwUVEHmfI)) for a brief overview of the `einsum` function and how it works. (You don't need to read past section 2.10.)


## Notes

* `einops` already supports fancy einsum

### Links

* [Einops tutorial - basics](https://einops.rocks/1-einops-basics/)

## Einops


In [3]:
import einops

In [4]:
import jaxtyping
from jaxtyping import UInt8, jaxtyped

from typeguard import typechecked as typechecker

In [5]:
# automatically apply jaxtyping?
%load_ext jaxtyping
%jaxtyping.typechecker typeguard.typechecked

In [6]:
# (6, 3, 150, 150)
arr: UInt8[np.ndarray, 'number channel height width'] = np.load(section_dir / "numbers.npy")

In [7]:
print(arr.shape)
print(arr.dtype)

(6, 3, 150, 150)
uint8


`arr` is a 4D numpy array. The first axes corresponds to the number, and the next three axes are channels (i.e. RGB), height and width respectively. You have the function `utils.display_array_as_img` which takes in a numpy array and displays it as an image. There are two possible ways this function can be run:

* If the input is three-dimensional, the dimensions are interpreted as `(channel, height, width)` - in other words, as an RGB image.
* If the input is two-dimensional, the dimensions are interpreted as `(height, width)` - i.e. a monochrome image.

For example:


In [8]:
# create wrapper functions with debugging
# TODO(bschoen): Are there named

# TODO(bschoen): Can't you concatenate here?
Image = UInt8[np.ndarray, 'height width']
RGBImage = UInt8[np.ndarray, 'channel height width']

# TODO(bschoen): This is really painful as it doesn't show the shapes of the incorrect argument @jaxtyped(typechecker=typechecker)
#
def display_array_as_image_debug(array: Image | RGBImage) -> None:

  print(f'{array.shape=}')

  jaxtyping.print_bindings()

  # call given function
  display_array_as_img(array)

In [10]:
# display_array_as_image_debug(arr)

A series of images follow below, which have been created using `einops` functions performed on `arr`. You should work through these and try to produce each of the images yourself. This page also includes solutions, but you should only look at them after you've tried for at least five minutes.

> *Note - if you find you're comfortable with the first ~half of these, you can skip to later sections if you'd prefer, since these aren't particularly conceptually important.*


### Einops exercises - images

```c
Difficulty: 🔴🔴⚪⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend up to ~45 minutes on these exercises collectively.

If you think you get the general idea, then you can skip to the next section.
```


#### Exercise 1


In [11]:
display_soln_array_as_img(1)

In [12]:
# Your code here - define arr1

# resulting dimensions are computed very simply
# length of newly composed axis is a product of components
# [6, 96, 96, 3] -> [96, (6 * 96), 3]
arr1: UInt8[np.ndarray, 'channel height width'] = einops.rearrange(arr, 'b c h w -> c h (b w)')

display_array_as_image_debug(arr1)

array.shape=(3, 150, 900)



<details>
<summary>Solution </summary>

```python
arr1 = einops.rearrange(arr, "b c h w -> c h (b w)")
```
</details>


---


#### Exercise 2


In [13]:
display_soln_array_as_img(2)

In [14]:
# Your code here - define arr2

# get 0
arr_zero: UInt8[np.ndarray, 'c h w'] = arr[0]

print(f'{arr_zero.shape=}')

# repeat it in the `h` direction
arr2 = einops.repeat(arr_zero, 'c h w -> c (2 h) w')

display_array_as_image_debug(arr2)

arr_zero.shape=(3, 150, 150)
array.shape=(3, 300, 150)



<details>
<summary>Solution </summary>

```python
arr2 = einops.repeat(arr[0], "c h w -> c (2 h) w")
```
</details>


---


#### Exercise 3


In [15]:
display_soln_array_as_img(3)

In [16]:
# Your code here - define arr3

# rearrange can also take care of lists of arrays with the same shape
# that's how we can stack inputs
# "list axis" becomes first ("b" in this case), and we left it there

# grab the 0 and the 1
arr_zero = arr[0]
arr_one = arr[1]

x = [arr_zero, arr_one]

print(type(x), 'with', len(x), 'tensors of shape', x[0].shape)

# - take new dimension `b`
# - and stack it in the `h` dimension
arr_zero_and_one = einops.rearrange(
    [arr_zero, arr_one],
    'b c h w -> c (b h) w'
)

print(f"{arr_zero_and_one.shape=}")

# now copy it one to the right
arr3 = einops.repeat(arr_zero_and_one, 'c h w -> c h (2 w)')

display_array_as_image_debug(arr3)

# display_array_as_img(arr3)

<class 'list'> with 2 tensors of shape (3, 150, 150)
arr_zero_and_one.shape=(3, 300, 150)
array.shape=(3, 300, 300)



<details>
<summary>Solution </summary>

```python
arr3 = einops.repeat(arr[0:2], "b c h w -> c (b h) (2 w)")
```
</details>


---


#### Exercise 4


In [17]:
display_soln_array_as_img(4)

In [18]:
# Your code here - define arr4

arr_zero = arr[0]

# make height twice as big
arr4 = einops.repeat(arr_zero, 'c h w -> c (h 2) w')

display_array_as_image_debug(arr4)

array.shape=(3, 300, 150)



<details>
<summary>Solution </summary>

```python
arr4 = einops.repeat(arr[0], "c h w -> c (h 2) w")
```
</details>


---


#### Exercise 5


In [19]:
display_soln_array_as_img(5)

In [20]:
# Your code here - define arr5
arr_zero = arr[0]

arr5 = einops.rearrange(arr_zero, 'c h w -> h (c w)')

display_array_as_image_debug(arr5)

array.shape=(150, 450)



<details>
<summary>Solution </summary>

```python
arr5 = einops.rearrange(arr[0], "c h w -> h (c w)")
```
</details>


---


#### Exercise 6


In [21]:
display_soln_array_as_img(6)

In [22]:
# Your code here - define arr6

arr_zero_one_two = arr[:3]
arr_three_four_five = arr[3:]

# stack them along the width direction
row_one = einops.rearrange(arr_zero_one_two, 'b c h w -> c h (b w)')
row_two = einops.rearrange(arr_three_four_five, 'b c h w -> c h (b w)')

display_array_as_image_debug(row_one)
display_array_as_image_debug(row_two)

# now stack the rows
arr6 = einops.rearrange([row_one, row_two], 'b c h w -> c (b h) w')

display_array_as_image_debug(arr6)

array.shape=(3, 150, 450)



array.shape=(3, 150, 450)



array.shape=(3, 300, 450)



<details>
<summary>Solution </summary>

```python
arr6 = einops.rearrange(arr, "(b1 b2) c h w -> c (b1 h) (b2 w)", b1=2)
```
</details>


---


#### Exercise 7


In [23]:
display_soln_array_as_img(7)

In [24]:
# Your code here - define arr7

arr7 = einops.rearrange(arr, 'b c h w -> c h (b w)')

arr7 = einops.reduce(arr7, 'c h w -> h w', 'max')

display_array_as_image_debug(arr7)

array.shape=(150, 900)



<details>
<summary>Solution </summary>

```python
arr7 = einops.reduce(arr.astype(float), "b c h w -> h (b w)", "max").astype(int)
```
</details>


---


#### Exercise 8


In [25]:
display_soln_array_as_img(8)

In [26]:
# Your code here - define arr8

# note: if an axis is not present, assume it has been reduced
arr8 = einops.reduce(arr.astype(float), 'b c h w -> h w', 'min').astype(int)

display_array_as_img(arr8)

<details>
<summary>Solution </summary>

```python
arr8 = einops.reduce(arr.astype(float), "b c h w -> h w", "min").astype(int)
```
</details>


---


#### Exercise 9


In [27]:
display_soln_array_as_img(9)

In [28]:
# Your code here - define arr9

arr9 = arr[1]

arr9 = einops.rearrange(arr9, 'c h w -> c w h')

display_array_as_image_debug(arr9)

array.shape=(3, 150, 150)



<details>
<summary>Solution </summary>

```python
arr9 = einops.rearrange(arr[1], "c h w -> c w h")
```
</details>


---


#### Exercise 10


In [29]:
display_soln_array_as_img(10)

In [31]:
# Your code here - define arr10
# note: wait hover shows shape?!
arr10 = einops.rearrange(arr6, 'c h w -> c (h 0.5) w')

display_array_as_img(arr10)

EinopsError:  Error while processing rearrange-reduction pattern "c h w -> c (h 0.5) w".
 Input tensor shape: (3, 300, 450). Additional info: {}.
 Expression may contain dots only inside ellipsis (...)

<details>
<summary>Solution </summary>

```python
arr10 = einops.reduce(arr, "(b1 b2) c (h h2) (w w2) -> c (b1 h) (b2 w)", "max", h2=2, w2=2, b1=2)
```
</details>


### Einops exercises - operations

Next, we have a series of functions which you should implement using `einops`. In the dropdown below these exercises, you can find solutions to all of them.

First, let's define some functions to help us test our solutions:

In [33]:
def assert_all_equal(actual: t.Tensor, expected: t.Tensor) -> None:
    assert actual.shape == expected.shape, f"Shape mismatch, got: {actual.shape}, (expected: {expected.shape})"
    assert (actual == expected).all(), f"Value mismatch, got: {actual} (expected: {expected})"
    print("Passed!")

def assert_all_close(actual: t.Tensor, expected: t.Tensor, rtol=1e-05, atol=0.0001) -> None:
    assert actual.shape == expected.shape, f"Shape mismatch, got: {actual.shape}, (expected: {expected.shape})"
    assert t.allclose(actual, expected, rtol=rtol, atol=atol), f"Value mismatch, got: {actual} (expected: {expected})"
    print("Passed!")

#### Exercise A.1 - rearrange (1)

In [34]:
import torch

def rearrange_1() -> t.Tensor:
    '''Return the following tensor using only torch.arange and einops.rearrange:

    [[3, 4],
     [5, 6],
     [7, 8]]
    '''
    return einops.rearrange(torch.arange(3, 9), '(h w) -> h w', h=3)


expected = t.tensor([[3, 4], [5, 6], [7, 8]])
assert_all_equal(rearrange_1(), expected)

Passed!


<details>
<summary>Solution</summary>

```python
def rearrange_1() -> t.Tensor:
    '''Return the following tensor using only torch.arange and einops.rearrange:

    [[3, 4],
     [5, 6],
     [7, 8]]
    '''
    return einops.rearrange(t.arange(3, 9), "(h w) -> h w", h=3, w=2)
```
</details>

---

#### Exercise A.2 - rearrange (2)

In [35]:
def rearrange_2() -> t.Tensor:
    '''Return the following tensor using only torch.arange and einops.rearrange:

    [[1, 2, 3],
     [4, 5, 6]]
    '''
    return einops.rearrange(torch.arange(1, 7), '(h w) -> h w', h=2)


assert_all_equal(rearrange_2(), t.tensor([[1, 2, 3], [4, 5, 6]]))

Passed!


<details>
<summary>Solution</summary>

```python
def rearrange_2() -> t.Tensor:
    '''Return the following tensor using only torch.arange and einops.rearrange:

    [[1, 2, 3],
     [4, 5, 6]]
    '''
    return einops.rearrange(t.arange(1, 7), "(h w) -> h w", h=2, w=3)
```
</details>

---

#### Exercise A.3 - rearrange (3)

In [36]:
def rearrange_3() -> t.Tensor:
    '''Return the following tensor using only torch.arange and einops.rearrange:

    [
      [
        [1], [2], [3], [4], [5], [6]
      ]
    ]
    '''
    return einops.rearrange(torch.arange(1, 7), '(c h w) -> c h w', c=1, h=6, w=1)


assert_all_equal(rearrange_3(), t.tensor([[[1], [2], [3], [4], [5], [6]]]))

Passed!


<details>
<summary>Solution</summary>

```python
def rearrange_3() -> t.Tensor:
    '''Return the following tensor using only torch.arange and einops.rearrange:

    [[[1], [2], [3], [4], [5], [6]]]
    '''
    return einops.rearrange(t.arange(1, 7), "a -> 1 a 1")
```

</details>

---

#### Exercise B.1 - temperature average


In [37]:
def temperatures_average(temps: t.Tensor) -> t.Tensor:
    '''Return the average temperature for each week.

    temps: a 1D temperature containing temperatures for each day.
    Length will be a multiple of 7 and the first 7 days are for the first week, second 7 days for the second week, etc.

    You can do this with a single call to reduce.
    '''
    assert len(temps) % 7 == 0

    return einops.reduce(temps, '(weeks days) -> weeks', 'mean', days=7)

print(torch.mean(torch.Tensor([71, 72, 70, 75, 71, 72, 70])))

temps = t.Tensor([71, 72, 70, 75, 71, 72, 70, 68, 65, 60, 68, 60, 55, 59, 75, 80, 85, 80, 78, 72, 83])
expected = t.tensor([71.5714, 62.1429, 79.0])
assert_all_close(temperatures_average(temps), expected)

tensor(71.5714)
Passed!


<details>
<summary>Solution</summary>

```python
def temperatures_average(temps: t.Tensor) -> t.Tensor:
    '''Return the average temperature for each week.

    temps: a 1D temperature containing temperatures for each day.
    Length will be a multiple of 7 and the first 7 days are for the first week, second 7 days for the second week, etc.

    You can do this with a single call to reduce.
    '''
    assert len(temps) % 7 == 0
    return einops.reduce(temps, "(h 7) -> h", "mean")
```

</details>

---

#### Exercise B.2 - temperature difference

In [38]:
def temperatures_differences(temps: t.Tensor) -> t.Tensor:
    '''For each day, subtract the average for the week the day belongs to.

    temps: as above
    '''
    assert len(temps) % 7 == 0

    avg_temps = temperatures_average(temps)

    avg_temp_per_week = einops.repeat(
        avg_temps,
        'weeks -> (weeks 7)',
    )

    return temps - avg_temp_per_week


expected = t.tensor(
    [
        -0.5714,
        0.4286,
        -1.5714,
        3.4286,
        -0.5714,
        0.4286,
        -1.5714,
        5.8571,
        2.8571,
        -2.1429,
        5.8571,
        -2.1429,
        -7.1429,
        -3.1429,
        -4.0,
        1.0,
        6.0,
        1.0,
        -1.0,
        -7.0,
        4.0,
    ]
)
actual = temperatures_differences(temps)
assert_all_close(actual, expected)

Passed!


In [39]:
torch.std?

<details>
<summary>Solution</summary>

```python
def temperatures_differences(temps: t.Tensor) -> t.Tensor:
    '''For each day, subtract the average for the week the day belongs to.

    temps: as above
    '''
    assert len(temps) % 7 == 0
    avg = einops.repeat(temperatures_average(temps), "w -> (w 7)")
	return temps - avg
```
</details>

---

#### Exercise B.3 - temperature normalized

In [42]:
def temperatures_normalized(temps: t.Tensor) -> t.Tensor:
    '''For each day, subtract the weekly average and divide by the weekly standard deviation.

    temps: as above

    Pass torch.std to reduce.
    '''
    temps_weekly_stddev = einops.reduce(
        temps,
        '(weeks days) -> weeks',
        torch.std,
        days=7,
    )

    temps_weekly_stddev = einops.repeat(temps_weekly_stddev, 'w -> (w 7)')

    return temperatures_differences(temps) / temps_weekly_stddev


expected = t.tensor(
    [
        -0.3326,
        0.2494,
        -0.9146,
        1.9954,
        -0.3326,
        0.2494,
        -0.9146,
        1.1839,
        0.5775,
        -0.4331,
        1.1839,
        -0.4331,
        -1.4438,
        -0.6353,
        -0.8944,
        0.2236,
        1.3416,
        0.2236,
        -0.2236,
        -1.5652,
        0.8944,
    ]
)
actual = temperatures_normalized(temps)
assert_all_close(actual, expected)

Passed!


<details>
<summary>Solution</summary>

```python
def temperatures_normalized(temps: t.Tensor) -> t.Tensor:
    '''For each day, subtract the weekly average and divide by the weekly standard deviation.

    temps: as above

    Pass torch.std to reduce.
    '''
	avg = einops.repeat(temperatures_average(temps), "w -> (w 7)")
	std = einops.repeat(einops.reduce(temps, "(h 7) -> h", t.std), "w -> (w 7)")
	return (temps - avg) / std
```
</details>

---

In [62]:
n = 0

tensor = torch.arange(0, n*n)

tensor = einops.rearrange(tensor, '(h w) -> h w', h=n)

print(tensor)

tensor_t = einops.rearrange(tensor, 'h w -> w h')

print(tensor_t)

identity = (tensor == tensor_t).type(tensor.dtype)

identity

ZeroDivisionError: integer division or modulo by zero

In [64]:
t.zeros((0, 0))

tensor([], size=(0, 0))

#### Exercise C - identity matrix

In [65]:
def identity_matrix(n: int) -> t.Tensor:
    '''Return the identity matrix of size nxn.

    Don't use torch.eye or similar.

    Hint: you can do it with arange, rearrange, and ==.
    Bonus: find a different way to do it.
    '''
    assert n >= 0

    if n == 0:
      return torch.zeros((0, 0))

    tensor = torch.arange(0, n*n)

    tensor = einops.rearrange(tensor, '(h w) -> h w', h=n)

    # transpose
    tensor_t = einops.rearrange(tensor, 'h w -> w h')

    # identity <-> transpose is equal
    identity = (tensor == tensor_t).type(tensor.dtype)

    return identity

assert_all_equal(identity_matrix(3), t.Tensor([[1, 0, 0], [0, 1, 0], [0, 0, 1]]))

# wtf?
assert_all_equal(identity_matrix(0), t.zeros((0, 0)))

Passed!
Passed!


In [68]:
n = 3

# ah clever, uses broadcasting?
einops.rearrange(t.arange(n), "i->i 1") == torch.arange(n)

tensor([[ True, False, False],
        [False,  True, False],
        [False, False,  True]])

<details>
<summary>Solution</summary>

```python
def identity_matrix(n: int) -> t.Tensor:
    '''Return the identity matrix of size nxn.

    Don't use torch.eye or similar.

    Hint: you can do it with arange, rearrange, and ==.
    Bonus: find a different way to do it.
    '''
    assert n >= 0
    return (einops.rearrange(t.arange(n), "i->i 1") == t.arange(n)).float()
```

</details>

---

In [90]:
n = 3

tensor = torch.Tensor([0.25, 0.3, 0.2, 0.1, 0.15])

print(tensor)

tensor = torch.cumsum(tensor, dim=0)

print(tensor)

# repeat the cumsum tensor `n` times, one for each sample
tensor = einops.repeat(tensor, 'p -> n p', n=n)

print(tensor)

# now we want to use rand to select one from each row
tensor_rand = einops.rearrange(torch.rand(n), 'n -> n 1')

print(tensor_rand)

# print(rand_value)

# then you keep going until it's greater than one of them, then the next one is your index?

# now the index of the first non-zero value is our index to sample
tensor_minus_rand = tensor - tensor_rand

print(tensor_minus_rand)

print(tensor_minus_rand > 0)

# max gives us the first `true` value as `max.indices`
torch.max(tensor_minus_rand > 0, dim=1).indices

tensor([0.2500, 0.3000, 0.2000, 0.1000, 0.1500])
tensor([0.2500, 0.5500, 0.7500, 0.8500, 1.0000])
tensor([[0.2500, 0.5500, 0.7500, 0.8500, 1.0000],
        [0.2500, 0.5500, 0.7500, 0.8500, 1.0000],
        [0.2500, 0.5500, 0.7500, 0.8500, 1.0000]])
tensor([[0.5084],
        [0.4891],
        [0.0206]])
tensor([[-0.2584,  0.0416,  0.2416,  0.3416,  0.4916],
        [-0.2391,  0.0609,  0.2609,  0.3609,  0.5109],
        [ 0.2294,  0.5294,  0.7294,  0.8294,  0.9794]])
tensor([[False,  True,  True,  True,  True],
        [False,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True]])


tensor([1, 1, 0])

#### Exercise D - sample distribution

In [91]:
def sample_distribution(probs: t.Tensor, n: int) -> t.Tensor:
    '''Return n random samples from probs, where probs is a normalized probability distribution.

    probs: shape (k,) where probs[i] is the probability of event i occurring.
    n: number of random samples

    Return: shape (n,) where out[i] is an integer indicating which event was sampled.

    Use torch.rand and torch.cumsum to do this without any explicit loops.

    Note: if you think your solution is correct but the test is failing, try increasing the value of n.
    '''
    assert abs(probs.sum() - 1.0) < 0.001
    assert (probs >= 0).all()
    # [0.25, 0.3, 0.2, 0.1, 0.15]
    #  ->
    # [0.25, 0.55, 0.75, 0.85, 1]

    tensor = torch.cumsum(probs, dim=0)

    # repeat the cumsum tensor `n` times, one for each sample
    tensor = einops.repeat(tensor, 'p -> n p', n=n)

    # now we want to use rand to select one from each row
    tensor_rand = einops.rearrange(torch.rand(n), 'n -> n 1')

    # then you keep going until it's greater than one of them, then the next one is your index?

    # now the index of the first non-zero value is our index to sample
    tensor_minus_rand = tensor - tensor_rand

    # max gives us the first `true` value as `max.indices`
    return torch.max(tensor_minus_rand > 0, dim=1).indices



n = 10000000
probs = t.tensor([0.05, 0.1, 0.1, 0.2, 0.15, 0.4])
freqs = t.bincount(sample_distribution(probs, n)) / n
assert_all_close(freqs, probs, rtol=0.001, atol=0.001)

Passed!


<details>
<summary>Solution</summary>

```python
def sample_distribution(probs: t.Tensor, n: int) -> t.Tensor:
    '''Return n random samples from probs, where probs is a normalized probability distribution.

    probs: shape (k,) where probs[i] is the probability of event i occurring.
    n: number of random samples

    Return: shape (n,) where out[i] is an integer indicating which event was sampled.

    Use torch.rand and torch.cumsum to do this without any explicit loops.

    Note: if you think your solution is correct but the test is failing, try increasing the value of n.
    '''
    assert abs(probs.sum() - 1.0) < 0.001
    assert (probs >= 0).all()
    return (t.rand(n, 1) > t.cumsum(probs, dim=0)).sum(dim=-1)
```
</details>

---


#### Exercise E - classifier accuracy

In [ ]:
def classifier_accuracy(scores: t.Tensor, true_classes: t.Tensor) -> t.Tensor:
    '''Return the fraction of inputs for which the maximum score corresponds to the true class for that input.

    scores: shape (batch, n_classes). A higher score[b, i] means that the classifier thinks class i is more likely.
    true_classes: shape (batch, ). true_classes[b] is an integer from [0...n_classes).

    Use torch.argmax.
    '''
    assert true_classes.max() < scores.shape[1]
    pass


scores = t.tensor([[0.75, 0.5, 0.25], [0.1, 0.5, 0.4], [0.1, 0.7, 0.2]])
true_classes = t.tensor([0, 1, 0])
expected = 2.0 / 3.0
assert classifier_accuracy(scores, true_classes) == expected

<details>
<summary>Solution</summary>

```python
def classifier_accuracy(scores: t.Tensor, true_classes: t.Tensor) -> t.Tensor:
    '''Return the fraction of inputs for which the maximum score corresponds to the true class for that input.

    scores: shape (batch, n_classes). A higher score[b, i] means that the classifier thinks class i is more likely.
    true_classes: shape (batch, ). true_classes[b] is an integer from [0...n_classes).

    Use torch.argmax.
    '''
    assert true_classes.max() < scores.shape[1]
    return (scores.argmax(dim=1) == true_classes).float().mean()
```
</details>

---

#### Exercise F.1 - total price indexing

In [ ]:
def total_price_indexing(prices: t.Tensor, items: t.Tensor) -> float:
    '''Given prices for each kind of item and a tensor of items purchased, return the total price.

    prices: shape (k, ). prices[i] is the price of the ith item.
    items: shape (n, ). A 1D tensor where each value is an item index from [0..k).

    Use integer array indexing. The below document describes this for NumPy but it's the same in PyTorch:

    https://numpy.org/doc/stable/user/basics.indexing.html#integer-array-indexing
    '''
    assert items.max() < prices.shape[0]
    pass


prices = t.tensor([0.5, 1, 1.5, 2, 2.5])
items = t.tensor([0, 0, 1, 1, 4, 3, 2])
assert total_price_indexing(prices, items) == 9.0

<details>
<summary>Solution</summary>

```python
def total_price_indexing(prices: t.Tensor, items: t.Tensor) -> float:
    '''Given prices for each kind of item and a tensor of items purchased, return the total price.

    prices: shape (k, ). prices[i] is the price of the ith item.
    items: shape (n, ). A 1D tensor where each value is an item index from [0..k).

    Use integer array indexing. The below document describes this for NumPy but it's the same in PyTorch:

    https://numpy.org/doc/stable/user/basics.indexing.html#integer-array-indexing
    '''
    assert items.max() < prices.shape[0]
    return prices[items].sum().item()
```
</details>

---

#### Exercise F.2 - gather 2D

In [ ]:
def gather_2d(matrix: t.Tensor, indexes: t.Tensor) -> t.Tensor:
    '''Perform a gather operation along the second dimension.

    matrix: shape (m, n)
    indexes: shape (m, k)

    Return: shape (m, k). out[i][j] = matrix[i][indexes[i][j]]

    For this problem, the test already passes and it's your job to write at least three asserts relating the arguments and the output. This is a tricky function and worth spending some time to wrap your head around its behavior.

    See: https://pytorch.org/docs/stable/generated/torch.gather.html?highlight=gather#torch.gather
    '''
    "TODO: YOUR CODE HERE"
    out = matrix.gather(1, indexes)
    "TODO: YOUR CODE HERE"
    return out


matrix = t.arange(15).view(3, 5)
indexes = t.tensor([[4], [3], [2]])
expected = t.tensor([[4], [8], [12]])
assert_all_equal(gather_2d(matrix, indexes), expected)
indexes2 = t.tensor([[2, 4], [1, 3], [0, 2]])
expected2 = t.tensor([[2, 4], [6, 8], [10, 12]])
assert_all_equal(gather_2d(matrix, indexes2), expected2)

<details>
<summary>Solution</summary>

```python
def gather_2d(matrix: t.Tensor, indexes: t.Tensor) -> t.Tensor:
    '''Perform a gather operation along the second dimension.

    matrix: shape (m, n)
    indexes: shape (m, k)

    Return: shape (m, k). out[i][j] = matrix[i][indexes[i][j]]

    For this problem, the test already passes and it's your job to write at least three asserts relating the arguments and the output. This is a tricky function and worth spending some time to wrap your head around its behavior.

    See: https://pytorch.org/docs/stable/generated/torch.gather.html?highlight=gather#torch.gather
    '''
	assert matrix.ndim == indexes.ndim
	assert indexes.shape[0] <= matrix.shape[0]
	out = matrix.gather(1, indexes)
	assert out.shape == indexes.shape
	return out
```
</details>

---

#### Exercise F.3 - total price gather

In [ ]:
def total_price_gather(prices: t.Tensor, items: t.Tensor) -> float:
    '''Compute the same as total_price_indexing, but use torch.gather.'''
    assert items.max() < prices.shape[0]
    pass


prices = t.tensor([0.5, 1, 1.5, 2, 2.5])
items = t.tensor([0, 0, 1, 1, 4, 3, 2])
assert total_price_gather(prices, items) == 9.0

<details>
<summary>Solution</summary>

```python
def total_price_gather(prices: t.Tensor, items: t.Tensor) -> float:
    '''Compute the same as total_price_indexing, but use torch.gather.'''
    assert items.max() < prices.shape[0]
    return prices.gather(0, items).sum().item()
```
</details>

---

#### Exercise G - indexing

In [ ]:
def integer_array_indexing(matrix: t.Tensor, coords: t.Tensor) -> t.Tensor:
    '''Return the values at each coordinate using integer array indexing.

    For details on integer array indexing, see:
    https://numpy.org/doc/stable/user/basics.indexing.html#integer-array-indexing

    matrix: shape (d_0, d_1, ..., d_n)
    coords: shape (batch, n)

    Return: (batch, )
    '''
    pass


mat_2d = t.arange(15).view(3, 5)
coords_2d = t.tensor([[0, 1], [0, 4], [1, 4]])
actual = integer_array_indexing(mat_2d, coords_2d)
assert_all_equal(actual, t.tensor([1, 4, 9]))
mat_3d = t.arange(2 * 3 * 4).view((2, 3, 4))
coords_3d = t.tensor([[0, 0, 0], [0, 1, 1], [0, 2, 2], [1, 0, 3], [1, 2, 0]])
actual = integer_array_indexing(mat_3d, coords_3d)
assert_all_equal(actual, t.tensor([0, 5, 10, 15, 20]))

<details>
<summary>Solution</summary>

```python
def integer_array_indexing(matrix: t.Tensor, coords: t.Tensor) -> t.Tensor:
    '''Return the values at each coordinate using integer array indexing.

    For details on integer array indexing, see:
    https://numpy.org/doc/stable/user/basics.indexing.html#integer-array-indexing

    matrix: shape (d_0, d_1, ..., d_n)
    coords: shape (batch, n)

    Return: (batch, )
    '''
    return matrix[tuple(coords.T)]
```

</details>

---

#### Exercise H.1 - batched logsumexp

In [ ]:
def batched_logsumexp(matrix: t.Tensor) -> t.Tensor:
    '''For each row of the matrix, compute log(sum(exp(row))) in a numerically stable way.

    matrix: shape (batch, n)

    Return: (batch, ). For each i, out[i] = log(sum(exp(matrix[i]))).

    Do this without using PyTorch's logsumexp function.

    A couple useful blogs about this function:
    - https://leimao.github.io/blog/LogSumExp/
    - https://gregorygundersen.com/blog/2020/02/09/log-sum-exp/
    '''
    pass


matrix = t.tensor([[-1000, -1000, -1000, -1000], [1000, 1000, 1000, 1000]])
expected = t.tensor([-1000 + math.log(4), 1000 + math.log(4)])
actual = batched_logsumexp(matrix)
assert_all_close(actual, expected)
matrix2 = t.randn((10, 20))
expected2 = t.logsumexp(matrix2, dim=-1)
actual2 = batched_logsumexp(matrix2)
assert_all_close(actual2, expected2)

<details>
<summary>Solution</summary>

```python
def batched_logsumexp(matrix: t.Tensor) -> t.Tensor:
    '''For each row of the matrix, compute log(sum(exp(row))) in a numerically stable way.

    matrix: shape (batch, n)

    Return: (batch, ). For each i, out[i] = log(sum(exp(matrix[i]))).

    Do this without using PyTorch's logsumexp function.

    A couple useful blogs about this function:
    - https://leimao.github.io/blog/LogSumExp/
    - https://gregorygundersen.com/blog/2020/02/09/log-sum-exp/
    '''
    C = matrix.max(dim=-1).values
	exps = t.exp(matrix - einops.rearrange(C, "n -> n 1"))
	return C + t.log(t.sum(exps, dim=-1))
```
</details>

---

#### Exercise H.2 - batched softmax

In [ ]:
def batched_softmax(matrix: t.Tensor) -> t.Tensor:
    '''For each row of the matrix, compute softmax(row).

    Do this without using PyTorch's softmax function.
    Instead, use the definition of softmax: https://en.wikipedia.org/wiki/Softmax_function

    matrix: shape (batch, n)

    Return: (batch, n). For each i, out[i] should sum to 1.
    '''
    pass


matrix = t.arange(1, 6).view((1, 5)).float().log()
expected = t.arange(1, 6).view((1, 5)) / 15.0
actual = batched_softmax(matrix)
assert_all_close(actual, expected)
for i in [0.12, 3.4, -5, 6.7]:
    assert_all_close(actual, batched_softmax(matrix + i))
matrix2 = t.rand((10, 20))
actual2 = batched_softmax(matrix2)
assert actual2.min() >= 0.0
assert actual2.max() <= 1.0
assert_all_equal(actual2.argsort(), matrix2.argsort())
assert_all_close(actual2.sum(dim=-1), t.ones(matrix2.shape[:-1]))

<details>
<summary>Solution</summary>

```python
def batched_softmax(matrix: t.Tensor) -> t.Tensor:
    '''For each row of the matrix, compute softmax(row).

    Do this without using PyTorch's softmax function.
    Instead, use the definition of softmax: https://en.wikipedia.org/wiki/Softmax_function

    matrix: shape (batch, n)

    Return: (batch, n). For each i, out[i] should sum to 1.
    '''
    exp = matrix.exp()
	return exp / exp.sum(dim=-1, keepdim=True)
```
</details>

---

#### Exercise H.3 - batched logsoftmax

In [ ]:
def batched_logsoftmax(matrix: t.Tensor) -> t.Tensor:
    '''Compute log(softmax(row)) for each row of the matrix.

    matrix: shape (batch, n)

    Return: (batch, n). For each i, out[i] should sum to 1.

    Do this without using PyTorch's logsoftmax function.
    For each row, subtract the maximum first to avoid overflow if the row contains large values.
    '''
    pass


matrix = t.arange(1, 6).view((1, 5)).float()
start = 1000
matrix2 = t.arange(start + 1, start + 6).view((1, 5)).float()
actual = batched_logsoftmax(matrix2)
expected = t.tensor([[-4.4519, -3.4519, -2.4519, -1.4519, -0.4519]])
assert_all_close(actual, expected)

<details>
<summary>Solution</summary>

```python
def batched_logsoftmax(matrix: t.Tensor) -> t.Tensor:
    '''Compute log(softmax(row)) for each row of the matrix.

    matrix: shape (batch, n)

    Return: (batch, n). For each i, out[i] should sum to 1.

    Do this without using PyTorch's logsoftmax function.
    For each row, subtract the maximum first to avoid overflow if the row contains large values.
    '''
    C = matrix.max(dim=1, keepdim=True).values
	return matrix - C - (matrix - C).exp().sum(dim=1, keepdim=True).log()
```
</details>

---

#### Exercise H.4 - batched cross entropy loss

In [ ]:
def batched_cross_entropy_loss(logits: t.Tensor, true_labels: t.Tensor) -> t.Tensor:
    '''Compute the cross entropy loss for each example in the batch.

    logits: shape (batch, classes). logits[i][j] is the unnormalized prediction for example i and class j.
    true_labels: shape (batch, ). true_labels[i] is an integer index representing the true class for example i.

    Return: shape (batch, ). out[i] is the loss for example i.

    Hint: convert the logits to log-probabilities using your batched_logsoftmax from above.
    Then the loss for an example is just the negative of the log-probability that the model assigned to the true class. Use torch.gather to perform the indexing.
    '''
    pass


logits = t.tensor([[float("-inf"), float("-inf"), 0], [1 / 3, 1 / 3, 1 / 3], [float("-inf"), 0, 0]])
true_labels = t.tensor([2, 0, 0])
expected = t.tensor([0.0, math.log(3), float("inf")])
actual = batched_cross_entropy_loss(logits, true_labels)
assert_all_close(actual, expected)

<details>
<summary>Solution</summary>

```python
def batched_cross_entropy_loss(logits: t.Tensor, true_labels: t.Tensor) -> t.Tensor:
    '''Compute the cross entropy loss for each example in the batch.

    logits: shape (batch, classes). logits[i][j] is the unnormalized prediction for example i and class j.
    true_labels: shape (batch, ). true_labels[i] is an integer index representing the true class for example i.

    Return: shape (batch, ). out[i] is the loss for example i.

    Hint: convert the logits to log-probabilities using your batched_logsoftmax from above.
    Then the loss for an example is just the negative of the log-probability that the model assigned to the true class. Use torch.gather to perform the indexing.
    '''
	assert logits.shape[0] == true_labels.shape[0]
	assert true_labels.max() < logits.shape[1]

    logprobs = batched_logsoftmax(logits)
    indices = einops.rearrange(true_labels, "n -> n 1")
    pred_at_index = logprobs.gather(1, indices)
    return - einops.rearrange(pred_at_index, "n 1 -> n")
```
</details>

---

#### Exercise I.1 - collect rows

In [ ]:
def collect_rows(matrix: t.Tensor, row_indexes: t.Tensor) -> t.Tensor:
    '''Return a 2D matrix whose rows are taken from the input matrix in order according to row_indexes.

    matrix: shape (m, n)
    row_indexes: shape (k,). Each value is an integer in [0..m).

    Return: shape (k, n). out[i] is matrix[row_indexes[i]].
    '''
    assert row_indexes.max() < matrix.shape[0]
    pass


matrix = t.arange(15).view((5, 3))
row_indexes = t.tensor([0, 2, 1, 0])
actual = collect_rows(matrix, row_indexes)
expected = t.tensor([[0, 1, 2], [6, 7, 8], [3, 4, 5], [0, 1, 2]])
assert_all_equal(actual, expected)

<details>
<summary>Solution</summary>

```python
def collect_rows(matrix: t.Tensor, row_indexes: t.Tensor) -> t.Tensor:
    '''Return a 2D matrix whose rows are taken from the input matrix in order according to row_indexes.

    matrix: shape (m, n)
    row_indexes: shape (k,). Each value is an integer in [0..m).

    Return: shape (k, n). out[i] is matrix[row_indexes[i]].
    '''
    assert row_indexes.max() < matrix.shape[0]
    return matrix[row_indexes]
```
</details>

---

#### Exercise I.2 - collect columns

In [ ]:
def collect_columns(matrix: t.Tensor, column_indexes: t.Tensor) -> t.Tensor:
    '''Return a 2D matrix whose columns are taken from the input matrix in order according to column_indexes.

    matrix: shape (m, n)
    column_indexes: shape (k,). Each value is an integer in [0..n).

    Return: shape (m, k). out[:, i] is matrix[:, column_indexes[i]].
    '''
    assert column_indexes.max() < matrix.shape[1]
    pass


matrix = t.arange(15).view((5, 3))
column_indexes = t.tensor([0, 2, 1, 0])
actual = collect_columns(matrix, column_indexes)
expected = t.tensor([[0, 2, 1, 0], [3, 5, 4, 3], [6, 8, 7, 6], [9, 11, 10, 9], [12, 14, 13, 12]])
assert_all_equal(actual, expected)

<details>
<summary>Solution</summary>

```python
def collect_columns(matrix: t.Tensor, column_indexes: t.Tensor) -> t.Tensor:
    '''Return a 2D matrix whose columns are taken from the input matrix in order according to column_indexes.

    matrix: shape (m, n)
    column_indexes: shape (k,). Each value is an integer in [0..n).

    Return: shape (m, k). out[:, i] is matrix[:, column_indexes[i]].
    '''
    assert column_indexes.max() < matrix.shape[1]
    return matrix[:, column_indexes]
```
</details>

---

## Einsum

Einsum is a very useful function for performing linear operations, which you'll probably be using a lot during this programme.

> Note - we'll be using the `einops.einsum` version of the function, which works differently to the more conventional `torch.einsum`:
>
> * `einops.einsum` has the arrays as the first arguments, and uses spaces to separate dimensions in the string.
> * `torch.einsum` has the string as its first argument, and doesn't use spaces to separate dimensions (each dim is represented by a single character).
>
> For instance, `torch.einsum("ij,i->j", A, b)` is equivalent to `einops.einsum(A, b, "i j, i -> j")`. (Note, einops doesn't care whether there are spaces either side of `,` and `->`, so you don't need to match this syntax exactly.)

Although there are many different kinds of operations you can perform, they are all derived from three key rules:

1. Repeating letters in different inputs means those values will be multiplied, and those products will be in the output.
    * For example, `M = einops.einsum(A, B, "i j, i j -> i j")` just corresponds to the elementwise product `M = A * B` (because $M_{ij}=A_{ij}B_{ij}$).
2. Omitting a letter means that the axis will be summed over.
    * For example, if `x` is a 2D array with shape `(I, J)`, then `einops.einsum(x, "i j -> i")` will be a 1D array of length `I` containing the row sums of `x` (we're summing along the `j`-index, i.e. across rows).
3. We can return the unsummed axes in any order.
    * For example, `einops.einsum(x, "i j k -> k j i")` does the same thing as `einops.rearrange(x, "i j k -> k j i")`.

*Note - the einops creators supposedly have plans to support shape rearrangement, e.g. with operations like `einops.einsum(x, y, "i j, j k l -> i (k l)")` (i.e. combining the features of rearrange and einsum), so we can all look forward to that day!*


### Einsum exercises

```c
Difficulty: 🔴🔴⚪⚪⚪
Importance: 🔵🔵🔵🔵⚪

You should spend up to 15-20 minutes on these exercises collectively.

If you think you get the general idea, then you can skip to the next section.
```

In the following exercises, you'll write simple functions using `einsum` which replicate the functionality of standard NumPy functions: trace, matrix multiplication, inner and outer products. We've also included some test functions which you should run.

Note - this version of einsum will require that you include `->`, even if you're summing to a scalar (i.e. the right hand side of your string expression is empty).


In [ ]:
def einsum_trace(mat: np.ndarray):
    '''
    Returns the same as `np.trace`.
    '''
    pass

def einsum_mv(mat: np.ndarray, vec: np.ndarray):
    '''
    Returns the same as `np.matmul`, when `mat` is a 2D array and `vec` is 1D.
    '''
    pass

def einsum_mm(mat1: np.ndarray, mat2: np.ndarray):
    '''
    Returns the same as `np.matmul`, when `mat1` and `mat2` are both 2D arrays.
    '''
    pass

def einsum_inner(vec1: np.ndarray, vec2: np.ndarray):
    '''
    Returns the same as `np.inner`.
    '''
    pass

def einsum_outer(vec1: np.ndarray, vec2: np.ndarray):
    '''
    Returns the same as `np.outer`.
    '''
    pass

tests.test_einsum_trace(einsum_trace)
tests.test_einsum_mv(einsum_mv)
tests.test_einsum_mm(einsum_mm)
tests.test_einsum_inner(einsum_inner)
tests.test_einsum_outer(einsum_outer)

<details>
<summary>Solution</summary>


```python
def einsum_trace(mat: np.ndarray):
    '''
    Returns the same as `np.trace`.
    '''
    # SOLUTION
    return einops.einsum(mat, "i i ->")

def einsum_mv(mat: np.ndarray, vec: np.ndarray):
    '''
    Returns the same as `np.matmul`, when `mat` is a 2D array and `vec` is 1D.
    '''
    # SOLUTION
    return einops.einsum(mat, vec, "i j, j -> i")

def einsum_mm(mat1: np.ndarray, mat2: np.ndarray):
    '''
    Returns the same as `np.matmul`, when `mat1` and `mat2` are both 2D arrays.
    '''
    # SOLUTION
    return einops.einsum(mat1, mat2, "i j, j k -> i k")

def einsum_inner(vec1: np.ndarray, vec2: np.ndarray):
    '''
    Returns the same as `np.inner`.
    '''
    # SOLUTION
    return einops.einsum(vec1, vec2, "i, i ->")

def einsum_outer(vec1: np.ndarray, vec2: np.ndarray):
    '''
    Returns the same as `np.outer`.
    '''
    # SOLUTION
    return einops.einsum(vec1, vec2, "i, j -> i j")
```
</details>
